In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
import numpy as np

2024-02-05 17:31:57.546602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 17:31:57.650183: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-05 17:31:57.655375: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-05 17:31:57.655390: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
data_g1 = pd.read_excel("/home/ubuntu/training/ayush/continual_learning/G1.xlsx")
data_g2 = pd.read_excel("/home/ubuntu/training/ayush/continual_learning/G2.xlsx")
data_g3 = pd.read_excel("/home/ubuntu/training/ayush/continual_learning/G3.xlsx")

data_g1.shape, data_g2.shape, data_g3.shape

((7356, 4), (6455, 4), (6277, 4))

In [3]:
data_g1.drop('Unnamed: 0', axis =1, inplace = True)
data_g2.drop('Unnamed: 0', axis =1, inplace = True)
data_g3.drop('Unnamed: 0', axis =1, inplace = True)

In [4]:
data_g2['text'] = data_g2['text'].astype(str)

In [5]:
data_g1['tags'] = data_g1['tags'].apply(lambda x: re.sub(r"[^a-zA-Z_]", " ", x))
data_g2['tags'] = data_g2['tags'].apply(lambda x: re.sub(r"[^a-zA-Z_]", " ", x))
data_g3['tags'] = data_g3['tags'].apply(lambda x: re.sub(r"[^a-zA-Z_]", " ", x))

In [6]:
data_g1['tags'] = data_g1['tags'].apply(lambda x: ', '.join(word.strip() for word in set(x.split())))
data_g2['tags'] = data_g2['tags'].apply(lambda x: ', '.join(word.strip() for word in set(x.split())))
data_g3['tags'] = data_g3['tags'].apply(lambda x: ', '.join(word.strip() for word in set(x.split())))

In [7]:
data_g3['text'].apply(lambda x: len(x.split())).describe()

count    6277.000000
mean       14.180819
std        13.108034
min         1.000000
25%         5.000000
50%        11.000000
75%        19.000000
max       245.000000
Name: text, dtype: float64

In [8]:
data_g1_train, data_g1_test = train_test_split(data_g1, test_size = 0.2, random_state = 42)
data_g2_train, data_g2_test = train_test_split(data_g2, test_size = 0.2, random_state = 42)
data_g3_train, data_g3_test = train_test_split(data_g3, test_size = 0.2, random_state = 42)
data_g1_train.shape, data_g1_test.shape, data_g2_train.shape, data_g2_test.shape, data_g3_train.shape, data_g3_test.shape

((5884, 3), (1472, 3), (5164, 3), (1291, 3), (5021, 3), (1256, 3))

In [9]:
data_g1_train_100 = data_g1_train.sample(n = 100, random_state = 42)
data_g2_train_100 = data_g2_train.sample(n = 100, random_state = 42)

In [10]:
def preprocess_data(data):
    max_words = 10000 
    max_length = 300  
 
    tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
    tokenizer.fit_on_texts(data['text'])
 
    sequences = tokenizer.texts_to_sequences(data['text'])
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
 
    mlb = MultiLabelBinarizer()
    encoded_tags = mlb.fit_transform(data['tags'].apply(lambda x: set(x.split(', '))))
 
    return {'text': padded_sequences, 'tags': encoded_tags}

In [11]:
def create_and_train_model(data, model_name, vocab_size, embedding_dim, max_length, num_classes):
    model = keras.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(data['text'], data['tags'], epochs=50)

    model.save(f'/home/ubuntu/training/ayush/continual_learning/models/{model_name}.h5')

    return model

In [12]:
data1_train = preprocess_data(data_g1_train)
data1_test = preprocess_data(data_g1_test)

data2_train = preprocess_data(data_g2_train)
data2_test = preprocess_data(data_g2_test)

data3_train = preprocess_data(data_g3_train)
data3_test = preprocess_data(data_g3_test)

data1_train_100 = preprocess_data(data_g1_train_100)
data2_train_100 = preprocess_data(data_g2_train_100)

In [13]:
data_g1_train.shape

(5884, 3)

In [14]:
# data1['text'][1]
data1_train['tags'][5872]

array([1, 0, 0, 0])

## Train Starting Model on data1

In [15]:
model1 = create_and_train_model(data1_train, 'model1', vocab_size = 10000, embedding_dim = 50, max_length = 300, num_classes = 4)

2024-02-05 17:33:25.594773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-02-05 17:33:25.596326: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-05 17:33:25.596346: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-0-71): /proc/driver/nvidia/version does not exist
2024-02-05 17:33:25.597242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
184/184 [==============================] - 1s 3ms/step - loss: 0.4085 - accuracy: 0.5802
Epoch 2/50
184/184 [==============================] - 0s 3ms/step - loss: 0.1964 - accuracy: 0.8404
Epoch 3/50
184/184 [==============================] - 0s 3ms/step - loss: 0.1082 - accuracy: 0.8975
Epoch 4/50
184/184 [==============================] - 0s 3ms/step - loss: 0.0613 - accuracy: 0.9210
Epoch 5/50
184/184 [==============================] - 0s 3ms/step - loss: 0.0341 - accuracy: 0.9308
Epoch 6/50
184/184 [==============================] - 0s 3ms/step - loss: 0.0195 - accuracy: 0.9320
Epoch 7/50
184/184 [==============================] - 0s 3ms/step - loss: 0.0120 - accuracy: 0.9354
Epoch 8/50
184/184 [==============================] - 0s 3ms/step - loss: 0.0075 - accuracy: 0.9356
Epoch 9/50
184/184 [==============================] - 0s 3ms/step - loss: 0.0054 - accuracy: 0.9359
Epoch 10/50
184/184 [==============================] - 0s 3ms/step - loss: 0.0039 - accuracy: 0.9368

## Performance on g1_test data

In [16]:
model1 = keras.models.load_model("/home/ubuntu/training/ayush/continual_learning/models/model1.h5")

In [17]:
prediction = model1.predict(data1_test['text'])

46/46 [==============================] - 0s 1ms/step


In [18]:
prediction.shape

(1472, 4)

In [19]:
tags_order = ['allergy_name', 'cancer', 'chronic_disease', 'treatment']

def evaluate_model(model, test_data, tags_order):
    predictions = model.predict(test_data['text'])

    binary_predictions = (predictions > 0.5).astype(int)

    true_labels = test_data['tags']

    f1_scores = []
    for i in range(len(tags_order)):
        tag_f1 = f1_score(true_labels[:, i], binary_predictions[:, i])
        print(f"F1 score for tag '{tags_order[i]}': {tag_f1}")
        f1_scores.append(tag_f1)
        
    print("\nOverall Classification Report:\n", classification_report(true_labels, binary_predictions, target_names=tags_order))

    return f1_scores

f1_scores_data1 = evaluate_model(model1, data1_test, tags_order)

46/46 [==============================] - 0s 1ms/step
F1 score for tag 'allergy_name': 0.08888888888888889
F1 score for tag 'cancer': 0.21729490022172948
F1 score for tag 'chronic_disease': 0.475448168355417
F1 score for tag 'treatment': 0.5465465465465464

Overall Classification Report:
                  precision    recall  f1-score   support

   allergy_name       0.29      0.05      0.09        38
         cancer       0.25      0.19      0.22       256
chronic_disease       0.49      0.46      0.48       659
      treatment       0.57      0.53      0.55       689

      micro avg       0.49      0.44      0.46      1642
      macro avg       0.40      0.31      0.33      1642
   weighted avg       0.48      0.44      0.46      1642
    samples avg       0.49      0.44      0.46      1642



/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Load model1 and train model2 on data1_test and data2_test

In [20]:
model1 = keras.models.load_model("/home/ubuntu/training/ayush/continual_learning/models/model1.h5")

# Freeze the layers of the loaded model
for layer in model1.layers:
    layer.trainable = False

# Add new layers for the second model
model2 = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=50, input_length=300),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])

In [21]:
# Compile the new model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Combine data2_train and 100 samples from data1_train
data2_train_combined = {
    'text': np.concatenate([data2_train['text'], data1_train_100['text']]),
    'tags': np.concatenate([data2_train['tags'], data1_train_100['tags']])
}

In [22]:
# Train the new model on data2_train_combined
model2.fit(data2_train_combined['text'], data2_train_combined['tags'], epochs=50)

Epoch 1/50
165/165 [==============================] - 1s 3ms/step - loss: 0.4262 - accuracy: 0.5737
Epoch 2/50
165/165 [==============================] - 0s 3ms/step - loss: 0.2470 - accuracy: 0.7893
Epoch 3/50
165/165 [==============================] - 0s 3ms/step - loss: 0.1492 - accuracy: 0.8611
Epoch 4/50
165/165 [==============================] - 0s 3ms/step - loss: 0.0928 - accuracy: 0.8955
Epoch 5/50
165/165 [==============================] - 0s 3ms/step - loss: 0.0549 - accuracy: 0.9168
Epoch 6/50
165/165 [==============================] - 0s 3ms/step - loss: 0.0332 - accuracy: 0.9248
Epoch 7/50
165/165 [==============================] - 0s 3ms/step - loss: 0.0209 - accuracy: 0.9271
Epoch 8/50
165/165 [==============================] - 0s 3ms/step - loss: 0.0133 - accuracy: 0.9301
Epoch 9/50
165/165 [==============================] - 0s 3ms/step - loss: 0.0090 - accuracy: 0.9316
Epoch 10/50
165/165 [==============================] - 0s 3ms/step - loss: 0.0072 - accuracy: 0.9301

In [23]:
model2.save('/home/ubuntu/training/ayush/continual_learning/models/model2.h5')

In [24]:
model2 = keras.models.load_model("/home/ubuntu/training/ayush/continual_learning/models/model2.h5")

### Evaluate the new model on combined data1_test and data2_test

In [25]:
data12_test_combined = {
    'text': np.concatenate([data1_test['text'], data2_test['text']]),
    'tags': np.concatenate([data1_test['tags'], data2_test['tags']])
}

In [26]:
evaluate_model(model2, data12_test_combined, tags_order)

87/87 [==============================] - 0s 1ms/step
F1 score for tag 'allergy_name': 0.05217391304347826
F1 score for tag 'cancer': 0.2045231071779744
F1 score for tag 'chronic_disease': 0.34609929078014184
F1 score for tag 'treatment': 0.4836997725549659

Overall Classification Report:
                  precision    recall  f1-score   support

   allergy_name       0.07      0.04      0.05        74
         cancer       0.20      0.21      0.20       485
chronic_disease       0.44      0.28      0.35      1287
      treatment       0.47      0.49      0.48      1292

      micro avg       0.40      0.35      0.38      3138
      macro avg       0.30      0.26      0.27      3138
   weighted avg       0.41      0.35      0.37      3138
    samples avg       0.40      0.35      0.37      3138



/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.05217391304347826,
 0.2045231071779744,
 0.34609929078014184,
 0.4836997725549659]

## Train model2 on data3_train, data1_train_100, data2_train_100

In [27]:
model2 = keras.models.load_model("/home/ubuntu/training/ayush/continual_learning/models/model2.h5")

# Freeze the layers of the loaded model
for layer in model2.layers:
    layer.trainable = False

# Create a new model (model3) for data3
model3 = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=50, input_length=300),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])

In [28]:
# Compile the new model
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Combine data3_train, data1_train_100, and data2_train_100
data3_train_combined = {
    'text': np.concatenate([data3_train['text'], data1_train_100['text'], data2_train_100['text']]),
    'tags': np.concatenate([data3_train['tags'], data1_train_100['tags'], data2_train_100['tags']])
}

In [29]:
# Train model3 on data3_train_combined
model3.fit(data3_train_combined['text'], data3_train_combined['tags'], epochs=50)

Epoch 1/50
164/164 [==============================] - 1s 3ms/step - loss: 0.4354 - accuracy: 0.5493
Epoch 2/50
164/164 [==============================] - 0s 3ms/step - loss: 0.2678 - accuracy: 0.7617
Epoch 3/50
164/164 [==============================] - 0s 3ms/step - loss: 0.1565 - accuracy: 0.8594
Epoch 4/50
164/164 [==============================] - 0s 3ms/step - loss: 0.0937 - accuracy: 0.8956
Epoch 5/50
164/164 [==============================] - 0s 3ms/step - loss: 0.0581 - accuracy: 0.9061
Epoch 6/50
164/164 [==============================] - 0s 3ms/step - loss: 0.0358 - accuracy: 0.9157
Epoch 7/50
164/164 [==============================] - 0s 3ms/step - loss: 0.0226 - accuracy: 0.9173
Epoch 8/50
164/164 [==============================] - 0s 3ms/step - loss: 0.0150 - accuracy: 0.9219
Epoch 9/50
164/164 [==============================] - 0s 3ms/step - loss: 0.0108 - accuracy: 0.9261
Epoch 10/50
164/164 [==============================] - 0s 3ms/step - loss: 0.0076 - accuracy: 0.9240

In [30]:
model3.save('/home/ubuntu/training/ayush/continual_learning/models/model3.h5')

In [31]:
model3 = keras.models.load_model("/home/ubuntu/training/ayush/continual_learning/models/model3.h5")

### Evaluate model3 on combined data1_test, data2_test and data3_test

In [32]:
data123_test_combined = {
    'text': np.concatenate([data1_test['text'], data2_test['text'], data3_test['text']]),
    'tags': np.concatenate([data1_test['tags'], data2_test['tags'], data3_test['tags']])
}

In [33]:
evaluate_model(model3, data123_test_combined, tags_order)

126/126 [==============================] - 0s 1ms/step
F1 score for tag 'allergy_name': 0.041884816753926704
F1 score for tag 'cancer': 0.1991561181434599
F1 score for tag 'chronic_disease': 0.41995714722987454
F1 score for tag 'treatment': 0.5244058267314081

Overall Classification Report:
                  precision    recall  f1-score   support

   allergy_name       0.05      0.03      0.04       116
         cancer       0.24      0.17      0.20       684
chronic_disease       0.50      0.36      0.42      1883
      treatment       0.51      0.54      0.52      1885

      micro avg       0.46      0.40      0.43      4568
      macro avg       0.32      0.28      0.30      4568
   weighted avg       0.45      0.40      0.42      4568
    samples avg       0.46      0.40      0.42      4568



/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.041884816753926704,
 0.1991561181434599,
 0.41995714722987454,
 0.5244058267314081]